# 数据预处理及降维、聚类

使用本地base，python=3.13

## 导入库

In [2]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import rdFingerprintGenerator   # for the fingerprint generation
import numpy as np  # for the dimensionality reduction process of the fingerprint datas
import os
import pandas as pd
from PIL import Image
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.ipython_useSVG=True

## 路径设置

In [3]:
work_dir = "./"
print(os.getcwd())
os.chdir(work_dir)

d:\Codes\sysusky_ai4s


## 读取文件得到分子mols对象列表

In [4]:
#设置SMILES的路径
smi_file = "./data/molecule.smi"
mol_ids = []
mols = []
invalid_mols = []

#打开该路径
with open(smi_file, "r") as f:
    for line in f.readlines():
        line = line.strip()
        parts = line.split(",")
        if parts[0] == "molecule_id":
            continue
        mol_id = parts[0].strip()
        smiles = parts[1].strip()
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            invalid_mols.append((mol_id, smiles))
            continue
        mol_ids.append(mol_id)
        mols.append(mol)

# 得到 mol_ids(分子序号), mols(分子), invalid_mols(无效分子)
print(len(mols))
print(mols)

200
[<rdkit.Chem.rdchem.Mol object at 0x0000023D7DA9D310>, <rdkit.Chem.rdchem.Mol object at 0x0000023D7DA9D2A0>, <rdkit.Chem.rdchem.Mol object at 0x0000023D7DA9D230>, <rdkit.Chem.rdchem.Mol object at 0x0000023D7DA9D460>, <rdkit.Chem.rdchem.Mol object at 0x0000023D7DA9D3F0>, <rdkit.Chem.rdchem.Mol object at 0x0000023D7DA9D4D0>, <rdkit.Chem.rdchem.Mol object at 0x0000023D7DA9D540>, <rdkit.Chem.rdchem.Mol object at 0x0000023D7DA9D380>, <rdkit.Chem.rdchem.Mol object at 0x0000023D7DA9D620>, <rdkit.Chem.rdchem.Mol object at 0x0000023D7DA9D5B0>, <rdkit.Chem.rdchem.Mol object at 0x0000023D7DA9D770>, <rdkit.Chem.rdchem.Mol object at 0x0000023D7DA9D700>, <rdkit.Chem.rdchem.Mol object at 0x0000023D7DA9D7E0>, <rdkit.Chem.rdchem.Mol object at 0x0000023D7DA9D8C0>, <rdkit.Chem.rdchem.Mol object at 0x0000023D7DA9D850>, <rdkit.Chem.rdchem.Mol object at 0x0000023D7DA9D690>, <rdkit.Chem.rdchem.Mol object at 0x0000023D7DA9D930>, <rdkit.Chem.rdchem.Mol object at 0x0000023D7DA9D9A0>, <rdkit.Chem.rdchem.Mol 

## 绘制分子图形

In [5]:

# 绘图
grid_img = Draw.MolsToGridImage(
    mols = mols, 
    molsPerRow = 20, 
    subImgSize = (500, 500), 
    legends = mol_ids, 
    maxMols=len(mols), 
    useSVG = True
)
# print(type(grid_img))

svg_data = grid_img.data
if isinstance(svg_data, bytes):
    svg_str = svg_data.decode('utf-8')
else:
    svg_str = svg_data

# with open("molecules.svg", "w", encoding = "utf-8") as f:
#     f.write(svg_str)

## 分子指纹和特征向量构造

返回了200*1000+的一个矩阵

In [6]:
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski, Crippen, GraphDescriptors
from rdkit.Chem import rdFingerprintGenerator

def calculate_molecular_features(mol):
    """计算全面的分子特征向量，接受Mol对象作为输入"""
    if mol is None:
        return np.nan * np.ones(1061)  # 返回NaN数组表示无效分子
    
    # 初始化特征数组
    features = []
    
    # 1. 使用推荐的Morgan指纹生成方法
    mfpgen = rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=1024)
    fp = mfpgen.GetFingerprint(mol)
    features.extend(list(fp))
    
    # 2. 物理化学性质
    physchem_features = [
        Descriptors.MolWt(mol),                         # 分子量
        Descriptors.MolLogP(mol),                        # 疏水性 (LogP)
        Descriptors.TPSA(mol),                           # 极性表面积
        Descriptors.NumValenceElectrons(mol),            # 价电子数
        Descriptors.NumRadicalElectrons(mol),            # 自由基电子数
        Descriptors.FractionCSP3(mol),                  # sp3杂化碳比例
        Descriptors.HeavyAtomCount(mol),                 # 重原子数
        Lipinski.NHOHCount(mol),                         # NH和OH基团数
        Lipinski.NOCount(mol),                           # N和O原子数
        Crippen.MolMR(mol),                              # 摩尔折射率
    ]
    features.extend(physchem_features)
    
    # 3. 氢键特征
    hbond_features = [
        Lipinski.NumHDonors(mol),                       # 氢键供体数
        Lipinski.NumHAcceptors(mol),                    # 氢键受体数
        Lipinski.NumHeteroatoms(mol),                   # 杂原子数
    ]
    features.extend(hbond_features)
    
    # 4. 环系统特征
    ring_features = [
        Lipinski.RingCount(mol),                        # 环总数
        Lipinski.NumAromaticRings(mol),                 # 芳香环数
        Lipinski.NumAliphaticRings(mol),                # 脂肪环数
        Lipinski.NumSaturatedRings(mol),               # 饱和环数
        Lipinski.NumHeterocycles(mol),                  # 杂环数
        GraphDescriptors.BalabanJ(mol),                 # Balaban指数
        GraphDescriptors.BertzCT(mol),                  # Bertz拓扑指数
    ]
    features.extend(ring_features)
    
    # 5. 电荷分布特征
    charge_features = [
        Descriptors.MaxPartialCharge(mol),              # 最大部分电荷
        Descriptors.MinPartialCharge(mol),              # 最小部分电荷
        Descriptors.MaxAbsPartialCharge(mol),           # 最大绝对部分电荷
        Descriptors.MinAbsPartialCharge(mol),           # 最小绝对部分电荷
    ]
    features.extend(charge_features)
    
    # 6. 立体化学特征
    stereo_features = [
        Descriptors.NumAtomStereoCenters(mol),          # 原子立体中心数
        Descriptors.NumUnspecifiedAtomStereoCenters(mol), # 未指定立体中心数
    ]
    features.extend(stereo_features)
    
    # 7. 药效团特征
    pharmacophore_features = [
        Lipinski.NumRotatableBonds(mol),                # 可旋转键数
        Lipinski.NumAmideBonds(mol),                    # 酰胺键数
        Descriptors.NumAromaticCarbocycles(mol),        # 芳香碳环数
        Descriptors.NumAromaticHeterocycles(mol),       # 芳香杂环数
        Descriptors.NumSaturatedCarbocycles(mol),       # 饱和碳环数
        Descriptors.NumSaturatedHeterocycles(mol),      # 饱和杂环数
    ]
    features.extend(pharmacophore_features)
    
    # 8. 自定义组合特征
    # 这些特征组合了多个基本属性，能更好地表征分子性质
    custom_features = [
        physchem_features[0] / 100,                     # 分子量/100
        physchem_features[1] * physchem_features[2],    # LogP * TPSA
        hbond_features[0] + hbond_features[1],          # 总氢键能力
        ring_features[0] / physchem_features[6],       # 环数/重原子数
        pharmacophore_features[0] / physchem_features[0] * 1000, # 可旋转键密度
    ]
    features.extend(custom_features)
    
    return np.array(features)

小测试

In [7]:
print(f"分子数量: {len(mols)}")  # 应该显示 200

分子数量: 200


## 生成分子指纹及拼接的特征向量，并将其转换为Numpy数组

In [8]:

# 生成分子指纹及拼接的特征向量，并将其转换为Numpy数组

fps = []

for mol in mols:
    fingerprint = calculate_molecular_features(mol)
    if fingerprint is not None:
        fps.append(fingerprint)

X = np.array(fps)
print(len(mols))
print("X.shape:", X.shape)
print(X[:3])

200
X.shape: (200, 1061)
[[ 0.          1.          0.         ...  7.          0.13333333
  18.39215758]
 [ 0.          1.          0.         ...  5.          0.11764706
  15.91374749]
 [ 0.          0.          0.         ...  6.          0.08695652
   8.49470921]]


## 数据保存
由于rdkit和umap不能同时跑，所以先保存再读取

In [11]:
np.save('molecular_fingerprints.npy', X)
print("已将分子指纹特征矩阵保存为 'molecular_fingerprints.npy'")


已将分子指纹特征矩阵保存为 'molecular_fingerprints.npy'
